## Installation and running of AP code base:
From ML Python notes: see "Current (AP project) startup"

see also folder: 'tm_AP_code'

In [1]:
%load_ext autoreload
%autoreload 2
import nltk; nltk.download('wordnet'); nltk.download('stopwords'); nltk.download('punkt')
import globals as g # tjm

# from voice_identifier.py  main:
from spacy_classifiers import main_voice_classifier, htmlise
from abstraction_scores import main_abstraction_scorer
from readability_scorer import main_readability_scorer
from final_output_nb import main_final_output_scorer
#from spacy_classifiers_copy import main_voice_classifier2 # , temp1


print('done') 

[nltk_data] Downloading package wordnet to /Users/tmurray/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tmurray/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/tmurray/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


ValueError: cymem.cymem.Pool has the wrong size, try recompiling. Expected 64, got 48

In [2]:
1

1

In [80]:
ip_file = 'input.csv'
result_file = 'result.csv'
debug_file =  'debug.csv'
model_type =  'en_core_web_lg'
DEBUG_ABSTRACTION_HIERARCHY = False 
WEIGHT_METRICS = True
ignore_prompt = False  
save_html = True # syntax graphs
html_file_dir='html'

df1 = main_voice_classifier(model_type, ip_file, ignore_prompt, save_html=save_html, file_dir=html_file_dir)
print('MVC:', df1.columns) # see file 
df2 = main_abstraction_scorer(df1, DEBUG_ABSTRACTION_HIERARCHY = DEBUG_ABSTRACTION_HIERARCHY)
print('MAS:', df2.columns) # see file 
df3 = main_readability_scorer(df2, model_type)
print('MRS:', df3.columns) # see file 
df4 = main_final_output_scorer(df3, result_file, debug_file, WEIGHT_METRICS = WEIGHT_METRICS)
print('MOS: done') # df4 returns nothing...

# >> see final files:  result.csv, debug.csv, weighted_debug.csv, weighted_result.csv

print('done')

Loaded models
Index(['UID', 'prompt', 'response', 'score', 'PassAct', 'surv_ogive',
       'survey_id', 'prompt_number', 'prompt_id'],
      dtype='object')
Is this 0? 0
MVC: Index(['UID', 'survey_id', 'prompt_number', 'prompt_id', 'prompt', 'response',
       'clauses_text_final', 'clauses_doc_final', 'voice', 'score', 'PassAct',
       'file_loc'],
      dtype='object')
MAS: Index(['UID', 'survey_id', 'prompt_number', 'prompt_id', 'prompt', 'response',
       'clauses_text_final', 'clauses_doc_final', 'voice', 'score', 'PassAct',
       'file_loc', 'abstraction_score', 'abstraction_score_normalized'],
      dtype='object')
1.0 0.0 1.0 0.0
MRS: Index(['UID', 'survey_id', 'prompt_number', 'prompt_id', 'prompt', 'response',
       'clauses_text_final', 'clauses_doc_final', 'voice', 'score', 'PassAct',
       'file_loc', 'abstraction_score', 'abstraction_score_normalized',
       'grading_level', 'reading_ease', 'reading_ease_normalized',
       'grading_level_normalized', 'textacy_doc',

In [71]:
# read in all data for what comes next

import pandas as pd
import xlrd  # pip install xlrd

the_folder = '/Users/tmurray/Documents/---Work/--Projects-Topics/StageLens_Project/_StageLens_RESEARCH/rubrics_project/'
#infiles
datafilepath = the_folder + '1941surv_w_scores_8.18copy.xlsx'

df_file = pd.read_excel(datafilepath, sheet_name="scores" ) 
df_file = df_file.fillna(value="()")
#df_file[df_file.r_protocol_id==4] # limit to genergal protocol

df_r_s = df_file.loc[:, ('c_clean_prompt','c_clean_response','c_score_clean_num','r_pasAct','r_surv_ogive','survey_id', 'prompt_number', 'prompt_id')]  # ('Response','scorex2')] # replaced all scorex2 with socre in file
df_r_s.columns =  ['prompt','response','score','PassAct','survOgive','survey_id', 'prompt_number', 'prompt_id'] # also want to try grouping by r_pasAct; or compbine by surv_ogive
df_r_s['UID'] = df_r_s.apply(lambda row: '{0:0{width}}'.format(row['survey_id'], width=5)+ '.' + '{0:0{width}}'.format(row['prompt_number'], width=3)+'.', axis=1)

print('done')



done


In [ ]:
# get all -ing words in database
import re

inglist = []

df1 = df_r_s # [1:100]

for i,r in df1.iterrows():
    for w in  re.findall(r'\w+', r['response']):
        if w[-3:] == "ing":
            #print(w)
            inglist = inglist + [w.lower()]
inglist = list(set(inglist))
inglist = sorted(inglist) # ,key=str.lower
print(len(inglist))
#print(inglist)

In [41]:
# code to find clauses with specific words:

pos_adjs =  ['my', 'your', 'his', 'her', 'our', 'your', 'their'] # 'its', (NOT 'her' noun)
pos_pronouns = ['mine', 'yours', 'his', 'hers', 'ours', 'yours', 'theirs'] 
getwords =["get", "seem", "feel", "gets", "seems", "feels", "got", "seemed", "felt"]

lookfor = set(pos_adjs + pos_pronouns) # set(getwords) # set(pos_adjs + pos_pronouns)

getrows = [] 
for i, r in df4.iterrows():
    # c = 0
    #print(i)
    if not(r.clause_num == 0):
        cl_set = set(r.clauses_text_final.split())
        if (cl_set & lookfor): # intersect
            getrows.append(str(i)+"="+r.clauses_text_final+"="+r.voice)  #

getrows
# copy paste output into excel file etc...


['01649.029.01=Dies I will go to her funeral=A_pron_x',
 '01650.027.03=have all my admiration=A_pron_x',
 '01813.007.03=how alien my constructions are to them=A_pron_x',
 '01813.007.04=how backward they often consider my ideas=A_pron_x',
 '01816.008.01=my desire=Undefined',
 '01816.008.04=that it takes away from my enjoyment of or full immersion in something=A_pron_x',
 '01823.033.04=oh that is Jumping around how sweet in my own skin=A_pron_x',
 '01823.036.03=when I close my eyes to look life s movement=A_pron_x',
 '01825.006.03=Finding my voice=A_pron_x',
 '01825.006.04=my creative expression feels good=A_pron_x',
 '01869.011.03=appreciating the good things in my life Restorative activities=A_pron_x',
 '01880.033.02=lose many of my competencies=A_pron_x',
 '01880.033.04=my behavior becomes very awkward=A_pron_x',
 '01889.013.04=yet manifesting our utter uniqueness=A_pron_x',
 '01889.029.01=and father find their full divine embrace within me=A_pron_x',
 '02069.003.06=what changes I wel

In [ ]:
# create original sample data

import pandas as pd

the_folder = '/Users/tmurray/Documents/---Work/--Projects-Topics/StageLens_Project/_StageLens_RESEARCH/rubrics_project/'
#infiles
datafilepath = the_folder + '1941surv_w_scores_8.18copy.xlsx'

df_file = pd.read_excel(datafilepath, sheet_name="scores" ) 
df_file = df_file.fillna(value="()")
df_file[df_file.r_protocol_id==4] # limit to genergal protocol

df_r_s = df_file.loc[:, ('c_clean_prompt','c_clean_response','c_score_clean_num','r_pasAct','r_surv_ogive','survey_id', 'prompt_number', 'prompt_id')]  # ('Response','scorex2')] # replaced all scorex2 with socre in file
df_r_s.columns =  ['prompt','response','score','PassAct','survOgive','survey_id', 'prompt_number', 'prompt_id'] # also want to try grouping by r_pasAct; or compbine by surv_ogive
df_r_s['UID'] = df_r_s.apply(lambda row: '{0:0{width}}'.format(row['survey_id'], width=5)+ '.' + '{0:0{width}}'.format(row['prompt_number'], width=3)+'.', axis=1)

df_sample = df_r_s[[(len(x)<200 and len(x)>15) for x in df_r_s['response']]]
df_sample = df_sample[[x !=1.0 for x in df_sample['score']]]  # remove 1.0

sample_size = 20 # per level
#df_sample.groupby('score').apply(lambda x: x.sample(min(len(x), sample_size))) # 
df_sampled = df_sample.groupby('score', group_keys=False).apply(lambda x: x.sample(n=sample_size, replace=False)) # , group_keys=False

#output in the python directory **
df_sampled.to_csv("input.csv", index = False)

df_sampled.head()